# Unicorn Investment Analysis - Data Validation and Preparation

## Introduction

We will use input validation and label encoding to prepare a dataset for analysis. These are fundamental techniques used in all types of data analysis, from simple linear regression to complex neural networks. 

## Step 1: Imports

In [4]:
# Import libraries and packages.
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt



### Load the dataset


In [28]:
# Run this cell so pandas displays all columns
pd.set_option('display.max_columns', None)

In [61]:
# RUN THIS CELL TO IMPORT YOUR DATA. 
companies = pd.read_csv('Modified_Unicorn_Companies.csv')

# Display the first five rows.
companies.head(5)

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG"
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita..."


## Step 2: Data cleaning


Begin by displaying the data types of the columns in `companies`.

In [62]:
# Display the data types of the columns.
companies.dtypes


Company             object
Valuation            int64
Date Joined         object
Industry            object
City                object
Country/Region      object
Continent           object
Year Founded         int64
Funding             object
Select Investors    object
dtype: object

### Modify the data types

Notice that the data type of the `Date Joined` column is an `object`&mdash;in this case, a string. Convert this column to `datetime` to make it more usable. 

In [63]:
# Apply necessary datatype conversions.
companies['Date Joined']=pd.to_datetime(companies['Date Joined'])


### Create a new column

We will add a column called `Years To Unicorn`, which is the number of years between when the company was founded and when it became a unicorn.

In [64]:
# Create the column Years To Unicorn.
companies['Years To Unicorn']=companies['Date Joined'].dt.year-companies['Year Founded']

our client interested in how quickly a company achieved unicorn status because that may revels certain trends or commonalities, and the client could use these information to find future companies to invest in. 

### Input validation

The data has some issues with bad data, duplicate rows, and inconsistent `Industry` labels.

We will identify and correct each of these issues.

#### Correcting bad data

Geting descriptive statistics for the `Years To Unicorn` column.

In [65]:
# Identify and correct the issue with Years To Unicorn.
companies['Years To Unicorn'].describe()

count    1074.000000
mean        7.013035
std         5.331842
min        -3.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        98.000000
Name: Years To Unicorn, dtype: float64

Isolating all rows where the `Years To Unicorn` column contains a negative value.

In [66]:
# Isolate any rows where `Years To Unicorn` is negative
companies[companies['Years To Unicorn']<0]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2020,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


- One company for internet software & services has negative values in the `Years To Unicorn` column.

An internet search reveals that InVision was founded in 2011. We will replace the value at `Year Founded` with `2011` for InVision's row. 

In [67]:
# Replace InVision's `Year Founded` value with 2011
companies.loc[companies['Company']=='InVision','Year Founded']=2011

# Verify the change was made properly
companies[companies['Company']=='InVision']


,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2011,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


Now, We will recalculate all the values in the `Years To Unicorn` column to remove the negative value for InVision. And verifying that there are no more negative values afterwards.

In [68]:
# Recalculate all values in the `Years To Unicorn` column
companies["Years To Unicorn"]=companies['Date Joined'].dt.year-companies['Year Founded']

# Verify that there are no more negative values in the column
companies['Years To Unicorn'].describe()


count    1074.000000
mean        7.021415
std         5.323155
min         0.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        98.000000
Name: Years To Unicorn, dtype: float64

#### Issues with `Industry` labels

The company provided us with the following list of industry labels to identify in the data for `Industry`. 


In [69]:
# List provided by the company of the expected industry labels in the data
industry_list = ['Artificial intelligence', 'Other','E-commerce & direct-to-consumer', 'Fintech',\
       'Internet software & services','Supply chain, logistics, & delivery', 'Consumer & retail',\
       'Data management & analytics', 'Edtech', 'Health', 'Hardware','Auto & transportation', \
        'Travel', 'Cybersecurity','Mobile & telecommunications']

In [70]:
# Check which values are in `Industry` but not in `industry_list`
set(companies['Industry'])-set(industry_list)

{'Artificial Intelligence', 'Data management and analytics', 'FinTech'}

- 'Artificial Intelligence', 'Data management and analytics', 'FinTech', they are misspelled.

Now, we will correct the bad entries in the `Industry` column by replacing them with an approved string from `industry_list`. To do this, we will use the [`replace()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html) `Series` method on the `Industry` series. When we pass a dictionary to the method, it will replace the data in the series where that data matches the dictionary's keys. The values that get imputed are the values of the dictionary. If a value is not specified in the dictionary, the series' original value is retained.
</br></br>

1. Creating a dictionary called `replacement_dict` whose keys are the incorrect spellings in the `Industry` series and whose values are the correct spelling, as indicated in `industry_list`.

2. Calling the `replace()` method on the `Industry` series and passing to it `replacement_dict` as its argument. Reassignimg the result back to the `Industry` column.

3. Verifying that there are no longer any elements in `Industry` that are not in `industry_list`.

In [71]:
# 1. Create `replacement_dict`
replacement_dict={'Artificial Intelligence':'Artificial intelligence','Data management and analytics':'Data management & analytics','FinTech':'Fintech'}

# 2. Replace the incorrect values in the `Industry` column
companies['Industry']=companies['Industry'].replace(replacement_dict)

# 3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`
set(companies['Industry'])-set(industry_list)

set()

#### Handling duplicate rows

The business mentioned that no company should appear in the data more than once.

We will verify that this is indeed the case, and if not, We will clean the data so each company appears only once.


In [89]:
# Isolate rows of all companies that have duplicates
companies[companies.duplicated(subset=['Company'], keep=False)]


,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Number,Country/Region Numeric,Artificial intelligence,Auto & transportation,Consumer & retail,Cybersecurity,Data management & analytics,E-commerce & direct-to-consumer,Edtech,Fintech,Hardware,Health,Internet software & services,Mobile & telecommunications,Other,"Supply chain, logistics, & delivery",Travel


The duplicated companies are not legitimate because they are clearly not different companies with the same name. They are the same company represented twice with minor variation.

In [74]:
# Drop rows of duplicate companies after their first occurrence
companies=companies.drop_duplicates(subset=['Company'],keep='first')

- It is important to perform input validation for ensuring data is complete, error-free, and high quality. A low-quality dataset may lead to an analysis that is incorrect or misleading.

**Steps we take to perform input validation for this dataset**


 * Fixing incorrect values
 * Correcting inconsistencies in the data
 * Removing duplicate data

### Convert numerical data to categorical data

#### Creating a `High Valuation` column

The data in the `Valuation` column represents how much money (in billions, USD) each company is valued at. We will use the `Valuation` column to create a new column called `High Valuation`. For each company, the value in this column should be `low` if the company is in the bottom 50% of company valuations and `high` if the company is in the top 50%.

In [76]:
# Create new `High Valuation` column
# Use qcut to divide Valuation into 'high' and 'low' Valuation groups
companies['High Valuation']=pd.qcut(companies['Valuation'],2,labels= ['High', 'Low'])


### Converting categorical data to numerical data

Three common methods for changing categorical data to numerical are:

1. Label encoding: order matters (ordinal numeric labels)
2. Label encoding: order doesn't matter (nominal numeric labels)
3. Dummy encoding: order doesn't matter (creation of binary columns for each possible category contained in the variable)

The decision on which method to use depends on the context and must be made on a case-to-case basis. However, a distinction is typically made between categorical variables with equal weight given to all possible categories vs. variables with a hierarchical structure of importance to their possible categories.  

For example, a variable called `subject` might have possible values of `history`, `mathematics`, `literature`. In this case, each subject might be **nominal**&mdash;given the same level of importance. However, we might have another variable called `class`, whose possible values are `freshman`, `sophomore`, `junior`, `senior`. In this case, the class variable is **ordinal**&mdash;its values have an ordered, hierarchical structure of importance. 

Machine learning models typically need all data to be numeric, and they generally use ordinal label encoding (method 1) and dummy encoding (method 3). 

In the next steps, we'll convert the following variables: `Continent`, `Country/Region`, and `Industry`, each using a different approach.

### Converting `Continent` to numeric

We will suppose that the investment group has specified that they want to give more weight to continents with fewer unicorn companies because they believe this could indicate unrealized market potential. 

This would make Continent an ordinal variable, since more importance is placed on continents with fewer unicorn companies. There is a hierarchy of importance.

Ranking the continents in descending order from the greatest number of unicorn companies to the least.

In [78]:
# Rank the continents by number of unicorn companies
companies['Continent'].value_counts()

North America    586
Asia             310
Europe           143
South America     21
Oceania            8
Africa             3
Name: Continent, dtype: int64

Now, we will create a new column called `Continent Number` that represents the `Continent` column converted to numeric in the order of their number of unicorn companies, where North America is encoded as `1`, through Africa, encoded as `6`.

In [90]:
# Create numeric `Continent Number` column

continent_dict = {'North America': 1,
                  'Asia': 2,
                  'Europe': 3,
                  'South America': 4,
                  'Oceania': 5,
                  'Africa': 6
                 }
companies['Continent Number']=companies['Continent'].replace(continent_dict)
companies.head(5)

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Number,Country/Region Numeric,Artificial intelligence,Auto & transportation,Consumer & retail,Cybersecurity,Data management & analytics,E-commerce & direct-to-consumer,Edtech,Fintech,Hardware,Health,Internet software & services,Mobile & telecommunications,Other,"Supply chain, logistics, & delivery",Travel
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,Low,2,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,Low,1,44,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,Low,2,9,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,Low,1,44,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,Low,3,38,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### Convert `Country/Region` to numeric

Now, we will suppose that within a given continent, each company's `Country/Region` is given equal importance. For analytical purposes, we want to convert the values in this column to numeric without creating a large number of dummy columns. We will use label encoding of this nominal categorical variable to create a new column called `Country/Region Numeric`, wherein each unique `Country/Region` is assigned its own number. 

In [82]:
# Create `Country/Region Numeric` column
# Create numeric categories for Country/Region
companies['Country/Region Numeric']=companies['Country/Region'].astype('category').cat.codes

### Convert `Industry` to numeric

Finally, we will create dummy variables for the values in the `Industry` column. 

In [85]:
# Convert `Industry` to numeric data
# Create dummy variables with Industry values
industry_encoded=pd.get_dummies(companies['Industry'])

# Combine `companies` DataFrame with new dummy Industry columns
companies=pd.concat([companies, industry_encoded], axis=1)



Displaying the first few rows of `companies`

In [86]:
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Number,Country/Region Numeric,Artificial intelligence,Auto & transportation,Consumer & retail,Cybersecurity,Data management & analytics,E-commerce & direct-to-consumer,Edtech,Fintech,Hardware,Health,Internet software & services,Mobile & telecommunications,Other,"Supply chain, logistics, & delivery",Travel
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,Low,2,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,Low,1,44,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,Low,2,9,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,Low,1,44,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,Low,3,38,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


**Categorical encoding approach we used for each variable**

* `Continent` - Ordinal label encoding was used because there was a hierarchical order to the categories.
* `Country/Region` - Nominal label encoding was used because there was not a hierarchical order the categories.
* `Industry` - Dummy encoding was used because there were not many different categories represented and they were all equally important.

## Conclusion

Input validation is essential for ensuring data is high quality and error-free.
In practice, input validation requires trial and error to identify issues and determine the best way to fix them.
There are benefits and disadvantages to both label encoding and dummy/one-hot encoding.
The decision to use label encoding versus dummy/one-hot encoding needs to be made on a case-by-case basis.

**Reference**

[Bhat, M.A. *Unicorn Companies*](https://www.kaggle.com/datasets/mysarahmadbhat/unicorn-companies)

